In [1]:
import sys
import os
from IPython.display import display, Markdown

sys.path.append(r'C:\Users\Pc\Documents\Talk_to_Eve_mvp')
from modules.global_configs import OPENAI_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [2]:
# Importa la biblioteca de OpenAI
from openai import OpenAI

def obtener_respuesta_gpt4(prompt, historial=[]):
    """
    Realiza una solicitud a la API de OpenAI utilizando el modelo GPT-4.
    
    Args:
        prompt (str): El texto de entrada o pregunta para el modelo.
    
    Returns:
        str: La respuesta generada por el modelo.
    """
    try:
        # Crea una instancia del cliente OpenAI
        cliente = OpenAI()

        messages=[{"role": "user", "content": prompt}]
        if len(historial) > 0:
            # xtend messages list role: changing roles user, agent. the none indexes are agent the pairs are user
            messages.extend([{"role": "user", "content": m} if i % 2 != 0 else {"role": "assistant", "content": m} for i, m in enumerate(historial)])

        # Realiza una solicitud de chat completions con el modelo GPT-4
        respuesta = cliente.chat.completions.create(
            model="gpt-4",  # Utiliza el modelo GPT-4
            messages=messages
        )

        # Devuelve la respuesta generada por el modelo
        return respuesta.choices[0].message
    except Exception as e:
        return f"Error al obtener respuesta: {str(e)}"


In [3]:
prompt = 'Estoy haciendo una api con fastapi y quiero saber como puedo manejar una secion de usuario, para impedir que cualquiera pueda usarla'

In [4]:
response = obtener_respuesta_gpt4(prompt)
display(Markdown(response.content))

FastAPI no viene con una capacidad integrada de manejo de sesión. Sin embargo, puedes manejar las sesiones de usuario y la autenticación utilizando diferentes métodos. Aquí hay algunas sugerencias:

**JWT (JSON Web Tokens):**
JWT es un estándar para crear tokens que afirman un conjunto de afirmaciones. Es comúnmente utilizado para autenticación y la información podría ser confiable porque está firmada digitalmente. La biblioteca `python-jose` es utilizada para trabajar con JWT en Python.

Ejemplo de cómo podría ser el código:

```python
from fastapi import FastAPI, Depends, HTTPException, status
from fastapi.security import OAuth2PasswordBearer, OAuth2PasswordRequestForm
from jose import JWTError, jwt
from passlib.context import CryptContext

# OAuth2 con contraseña (y hashing), Bearer con JWT token
oauth2_scheme = OAuth2PasswordBearer(tokenUrl="token")

@app.post("/token", response_model=Token)
def login(form_data: OAuth2PasswordRequestForm = Depends()):
    u = authenticate_fake(form_data.username, form_data.password)
    # ...
    access_token_expires = timedelta(minutes=ACCESS_TOKEN_EXPIRE_MINUTES)
    access_token = create_access_token(
        data={"sub": u.username}, expires_delta=access_token_expires
    )
    return {"access_token": access_token, "token_type": "bearer"}

@app.get("/users/me")
def read_users_me(token: str = Depends(oauth2_scheme)):
    # aquí podrías hacer la lógica de verificación del token
```

**Uso de un Middleware para manejar las sesiones:**

A continuación, se muestra un código de ejemplo en el que se utiliza un middleware personalizado para manejar las sesiones.

```python
from starlette.middleware.sessions import SessionMiddleware
from starlette.sessions import Session
from fastapi import FastAPI, Depends, HTTPException

app = FastAPI()

app.add_middleware(SessionMiddleware, secret_key="very-secret-key", session_cookie="my-cookie")

def get_session(session: Session = Depends(Session)):
    user_id = session.get("user_id")
    if not user_id:
        raise HTTPException(status_code=403, detail="Unauthorized")
    return "Hello, User"

@app.get("/private-area")
def get_private_area(session=Depends(get_session)):
    return session
```

En el código de arriba, primero agregamos un middleware `SessionMiddleware`. Con FastAPI, el middleware se ejecuta antes de que se procese una solicitud y puede manipularla de acuerdo a sus necesidades. Luego declaramos la dependencia `Session` que busca la cookie de sesión en la solicitud y devuelve un diccionario (Session).

Entonces, puedes manejar las sesiones de los usuarios y restringir el acceso a ciertas rutas comprobando si el user_id existe en la sesión.

Además de estos métodos, también puedes usar sistemas de autenticación de terceros como OAuth2 y OpenID Connect, que se pueden integrar con FastAPI.

### documentation

In [5]:
prompt = """    i need you explain how to use my fastapi. can you explain really clearlly with examples of imputs and outputs for each endpoint? If you do it well i will give you a big reward.
        
                from fastapi import FastAPI, HTTPException, Response, Depends, UploadFile, File
                import base64
                from pydantic import BaseModel
                import sys
                import os
                from typing import Union, List, Dict
                import shutil
                import pandas as pd
                from fastapi.middleware.cors import CORSMiddleware
                from modules.users_manager.user_manager import UserManager_v1
                import ssl

                current_dir = os.path.dirname(os.path.abspath(__file__))
                # Obtain the parent directory
                parent_dir = os.path.dirname(current_dir)
                grandparent_dir = os.path.dirname(parent_dir)
                sys.path.append(parent_dir)
                sys.path.append(grandparent_dir)

                from modules.chatbot.eves import Eve_v6

                app = FastAPI()

                origins = ["*"]

                app.add_middleware(
                    CORSMiddleware,
                    allow_origins=origins,
                    allow_credentials=True,
                    allow_methods=["*"],
                    allow_headers=["*"],
                )

                ssl_context = ssl.SSLContext(ssl.PROTOCOL_TLS_SERVER)
                ssl_context.load_cert_chain('app/cert.pem', keyfile='app/key.pem')

                user_eve_instances = {}

                class sign_in(BaseModel):
                    username: str 

                @app.post("/initialize_eve")
                def initialize_eve(payload: sign_in = Depends()):
                    if payload.username in user_eve_instances:
                        raise HTTPException(status_code=400, detail="Eve is already initialized for this user")

                    # Crear una nueva instancia de Eve_v6 para el usuario
                    user_eve_instances[payload.username] = Eve_v6()
                    user_eve_instances[payload.username].initialize(payload.username)
                    return {"response": "Eve initialized"}

                @app.post("/sign_up")
                def sign_up(payload: sign_in = Depends()):
                    user_manager = UserManager_v1(payload.username)
                    #load csv of users
                    df = user_manager.user_list
                    #check if user already exists
                    if payload.username in df['user'].values:
                        raise HTTPException(status_code=400, detail="User already exists")
                    #add the user to the csv
                    user_manager.create_edit_user(payload.username)
                    del user_manager
                    return {"response": "User created"}

                class PatientMessage(BaseModel):
                    username: str
                    message: Union[str, None] = None
                    audio: Union[UploadFile, None] = None
                    audio_response_flag: bool = True 

                @app.post("/response_text")
                async def response_text(payload: PatientMessage = Depends()):
                    audio_flag = False
                    audio_response_path = None
                    
                    if payload.message is None and payload.audio is None:
                        raise HTTPException(status_code=400, detail="Message is empty")
                    if not payload.username in user_eve_instances:
                        raise HTTPException(status_code=400, detail="Eve is not initialized for this user")
                    
                    eve = user_eve_instances[payload.username]
                    
                    if payload.audio:
                        audio_flag = True
                        audio_path = os.path.join(current_dir, "TEMP", eve.username, "audio", payload.audio.filename)
                        os.makedirs(os.path.dirname(audio_path), exist_ok=True)
                        with open(audio_path, "wb") as f:
                            f.write(payload.audio.file.read())

                    eve_message, audio_response_path = eve.response_with_audio(payload.message, audio_flag=audio_flag, audio_response_flag=payload.audio_response_flag)
                    
                    if payload.audio_response_flag:
                        with open(audio_response_path, "rb") as f:
                            audio_bytes = f.read()
                        # audio_response = FileResponse(audio_response_path, media_type="audio/mpeg")
                        audio_base64 = base64.b64encode(audio_bytes).decode("utf-8")
                        return {"response": eve_message, "audio_b64": audio_base64}

                    return {"response": eve_message, "audio_b64": None}

                class indicators(BaseModel):
                    username: str
                    session_id: Union[str, None] = None
                    from_date: Union[str, None] = None
                    to_date: Union[str, None] = None
                    aggregate: bool = False

                @app.post("/indicators")
                def indicators(payload: indicators = Depends()):
                    if not payload.username in user_eve_instances:
                        raise HTTPException(status_code=400, detail="Eve is not initialized for this user")
                    
                    eve = user_eve_instances[payload.username]
                    
                    indicators = eve.user_manager.get_indicators(session_id = payload.session_id, from_date = payload.from_date, to_date = payload.to_date, aggregate = payload.aggregate)
                    
                    return indicators.to_dict('records')

                class keyphrases(BaseModel):
                    username: str
                    session_id: Union[str, None] = None
                    from_date: Union[str, None] = None
                    to_date: Union[str, None] = None

                @app.post("/keyphrases")
                def keyphrases(payload: keyphrases = Depends()):
                    if not payload.username in user_eve_instances:
                        raise HTTPException(status_code=400, detail="Eve is not initialized for this user")
                    
                    eve = user_eve_instances[payload.username]
                    
                    keyphrases = eve.user_manager.get_keyphrases(session_id = payload.session_id, from_date = payload.from_date, to_date = payload.to_date)
                    
                    return keyphrases.to_dict('records')

                class todo_survey_(BaseModel):
                    username: str
                @app.post("/todo_survey")
                def todo_survey(payload: todo_survey_ = Depends()):
                    if not payload.username in user_eve_instances:
                        raise HTTPException(status_code=400, detail="Eve is not initialized for this user")
                    
                    eve = user_eve_instances[payload.username]
                    
                    survey_df = eve.survey_db.get_survey_df()

                    return survey_df.to_dict('records')

                class NextQuestion(BaseModel):
                    username: str
                    survey_id: int
                    number_of_questions: int = 1

                @app.post("/survey_next_questions")
                def survey_next_questions(payload: NextQuestion = Depends()):
                    if not payload.username in user_eve_instances:
                        raise HTTPException(status_code=400, detail="Eve is not initialized for this user")
                    
                    eve = user_eve_instances[payload.username]
                    
                    survey_formatted = eve.user_manager.database.get_survey_next_questions(survey_id = payload.survey_id, number_of_questions = payload.number_of_questions)
                    # question_dicts = eve.obtain_question(question_id = None, return_dict = True, n_questions = payload.number_of_questions)
                    
                    return {"questions": survey_formatted}
                class Response(BaseModel):
                    username: str
                    survey_id: int
                    responses: Dict[int, int]

                @app.post("/response_survey")
                def response_survey(payload: Response):
                    if not payload.username in user_eve_instances:
                        raise HTTPException(status_code=400, detail="Eve is not initialized for this user")
                    
                    eve = user_eve_instances[payload.username]
                    eve.user_manager.database.insert_survey_response(payload)    

                    return {"response": "Survey response saved"}

                class finalize_eve_(BaseModel):
                    username: str
                @app.post("/finalize_eve")
                def finalize_eve(payload: finalize_eve_ = Depends()):
                    if not payload.username in user_eve_instances:
                        raise HTTPException(status_code=400, detail="Eve is not initialized for this user")
                    
                    eve = user_eve_instances.pop(payload.username)
                    
                    # delete temporary files
                    eve.finalize()
                    shutil.rmtree(os.path.join(current_dir, "TEMP", eve.username))
                    # delete eve object
                    del eve
                    # create new eve object
                    return {"response": "Eve finalized"}





"""

In [8]:
print(response.content)

This FastAPI application provides multiple endpoints for handling chatbot related service known as `Eve_v6`. Client can initialize the instance of Eve by registering user, get text responses from Eve, get keyphrases, and many more. Each of these functionalities are exposed via specific endpoints mentioned in the application.

1. **Endpoint: `/initialize_eve`**
   - Method: POST
   - Purpose: This endpoint is used to initialize an instance of Eve with the provided username.
   - Request example:
     ```
       { "username": "John" }
     ```
   - Output example:
     ```
       { "response": "Eve initialized" }
     ```
     
2. **Endpoint: `/sign_up`**
   - Method: POST
   - Purpose: To register a new username. If the user already exists, it will throw an HTTPException.
   - Request example:
     ```
       { "username": "John" }
     ```
   - Output example:
     ```
       { "response": "User created" }
     ```

3. **Endpoint: `/response_text`**
   - Method: POST
   - Purpose: This

In [6]:
response = obtener_respuesta_gpt4(prompt)
display(Markdown(response.content))

This FastAPI application provides multiple endpoints for handling chatbot related service known as `Eve_v6`. Client can initialize the instance of Eve by registering user, get text responses from Eve, get keyphrases, and many more. Each of these functionalities are exposed via specific endpoints mentioned in the application.

1. **Endpoint: `/initialize_eve`**
   - Method: POST
   - Purpose: This endpoint is used to initialize an instance of Eve with the provided username.
   - Request example:
     ```
       { "username": "John" }
     ```
   - Output example:
     ```
       { "response": "Eve initialized" }
     ```
     
2. **Endpoint: `/sign_up`**
   - Method: POST
   - Purpose: To register a new username. If the user already exists, it will throw an HTTPException.
   - Request example:
     ```
       { "username": "John" }
     ```
   - Output example:
     ```
       { "response": "User created" }
     ```

3. **Endpoint: `/response_text`**
   - Method: POST
   - Purpose: This endpoint is used to generate a text response for the given input from Eve. It also handles audio input.
   - Inputs:
     ```
       { "username": "John",
         "message": "Hi there",
         "audio": (a file can be uploaded here with the field name as 'audio'),
         "audio_response_flag": true
       }
     ```
   - Output: 
     ```
       { "response": "Hello, John!",
         "audio_b64": "(base64 encoded audio response)"
       }
     ```

4. **Endpoint: `/indicators`**
   - Method: POST
   - Purpose: This endpoint is used to get indicators.
   - Requests example:
     ```
       {
        "username": "John",
        "session_id": "123",
        "from_date": "2022-03-01",
        "to_date": "2022-03-30",
        "aggregate": false
       }
     ```
   - Output example:
     ```
       [{'sessionId': 'xxxxxx', 'indicators': {...}}, {...}, ...]
     ```

5. **Endpoint: `/keyphrases`**
   - Method: POST
   - Purpose: To get keyphrases.
   - Requests example:
     ```
       {"username": "John","session_id": "123","from_date": "2022-03-01","to_date": "2022-03-30"}
     ```
   - Output example:
     ```
       [{'sessionId': 'xxxxxx', 'keyphrases': {...}}, {...}, ...]
     ```

6. **Endpoint: `/todo_survey`**
   - Method: POST
   - Purpose: To get list of survey.
   - Request example:
     ```
       { "username": "John" }
     ```
   - Output example:
     ```
       [{'surveyId': 'xxxxxx', 'questions': {...}}, {...}, ...]
     ```
     
7. **Endpoint: `/survey_next_questions`**
   - Method: POST
   - Purpose:  Retrieve the next set of questions for a specific survey.
   - Request example:
     ```
       { "username": "John", "survey_id": 1, "number_of_questions": 1 }
     ```
   - Output example:
     ```
       { "questions": "What is your favorite fruit?" }
     ```

8. **Endpoint: `/response_survey`**
   - Method: POST
   - Purpose: Stores the response of the user for a specific survey.
   - Request example:
     ```
       {"username": "John", "survey_id": 1, "responses": {"1": 1, "2": 2, "3": 3}}
     ```
   - Output example:
     ```
       { "response": "Survey response saved" }
     ```

9. **Endpoint: `/finalize_eve`**
   - Method: POST
   - Purpose: To terminate an active Eve instance and delete user associated temporary files.
   - Request example:
     ```
       { "username": "John" }
     ```
   - Output example:
     ```
       { "response": "Eve finalized" }
     ```

Please replace "John", "123" and other example values with your actual data. For file uploads like audio, you might need to use a form-data POST where you upload the file as binary and the rest of the fields as form data. 

FastAPI Swagger UI provides an easy interface for you to test these APIs. Just start your FastAPI service and navigate to `http://localhost:8000/docs` in your web browser (do replace `http://localhost:8000` with your actual FastAPI host and port).

In [72]:
prompt2 = """
        como harias una base de datos sqlite del siguiente json:
        {"id": "000", "name": "anonymous health and wellbeing survey y2 2023", "date": "05-12-2023", "question_0": {"question": "What is your position?", "answers": ["Officer / Chief / SL", "Operator / Rating", "Other"]}, "question_1": {"question": "What is your age?", "answers": ["Under 24", "25 - 34", "35 - 44", "45 - 54", "Over 55"]}, "question_2": {"question": "What shift are you on?", "answers": ["00 - 12", "12 - 24", "Dayshift", "Other"]}, "question_3": {"question": "How many years of seismic offshore experience do you have?", "answers": ["1 year or less", "2 - 4 years", "5 - 9 years", "10 - 15 years", "Over 15 years"]}, "question_4": {"question": "Which vessel do you work on?", "answers": ["Tethys", "Sovereign", "Vanguard", "Atlas", "Hyperion", "Tansa", "Apollo", "Swift", "Victory"]}, "question_5": {"question": "How many hours per day (on - and off shift) do you spend sitting orphysically inactive?", "answers": ["Less than 2 hrs", "3 - 4 hrs", "5 - 7 hrs", "8 - 11 hrs", "Over 11 hrs"]}, "question_6": {"question": "How often do you go outside to get some fresh air during a day?", "answers": ["Never", "Almost never", "Sometimes", "Fairly often", "Very often"]}, "question_7": {"question": "How often do you do exercise in your time off per week?", "answers": ["Never", "0 - 60 minutes", "61 - 100 minutes", "101 - 149 minutes", "150 - 300 minutes", "More than 300 minutes"]}, "question_8": {"question": "How many servings of fruit and vegetables do you usually eat per day (athome and at work)?", "answers": ["1 or less", "2 servings", "3 servings", "4 servings", "5 or more"]}, "question_9": {"question": "How many days a week do you usually eat fast food or food that is highin fat and salt (at home and at work)?", "answers": ["None", "1", "2 - 3 times", "4 - 5 times", "5 or more"]}, "question_10": {"question": "Healthy options that I want to eat are served onboard?", "answers": ["Never", "Almost never", "Sometimes", "Fairly often", "Very often"]}, "question_11": {"question": "When was the last time you had your cholesterol measured?", "answers": ["Never", "Last month", "6 months", "1 year", "Over 2 years"]}, "question_12": {"question": "When was the last time you had your blood pressuremeasured?", "answers": ["Last month", "6 months", "1 year", "Over 2 years"]}, "question_13": {"question": "I\u2019m comfortable seeing the Medic if I feel unwell.", "answers": ["Strongly agree", "Agree", "Neutral", "Disagree", "Strongly disagree"]}, "question_14": {"question": "In the last month, how often have you felt that you were unable to control the important things in your life?", "answers": ["Never", "Almost never", "Sometimes", "Fairly often", "Very often"]}, "question_15": {"question": "In the last month, how often have you felt confident about your ability to handle your personal problems?", "answers": ["Never", "Almost never", "Sometimes", "Fairly often", "Very often"]}, "question_16": {"question": "In the last month, how often have you felt that things were going yourway?", "answers": ["Never", "Almost never", "Sometimes", "Fairly often", "Very often"]}, "question_17": {"question": "In the last month, how often have you felt difficulties were piling up sohigh that you could not overcome them?", "answers": ["Never", "Almost never", "Sometimes", "Fairly often", "Very often"]}, "question_18": {"question": "How often are you able to stay in touch with your family and friends while offshore?", "answers": ["Never", "Almost never", "Sometimes", "Fairly often", "Very often"]}, "question_19": {"question": "How often do you socialize with you colleagues off shift?", "answers": ["Never", "Almost never", "Sometimes", "Fairly often", "Very often"]}, "question_20": {"question": "Welfare and sports offerings onboard are good?", "answers": ["Strongly agree", "Agree", "Neutral", "Disagree", "Strongly disagree"]}, "question_21": {"question": "I am comfortable seeing the Medic if I feel distressed or down.", "answers": ["Strongly agree", "Agree", "Neutral", "Disagree", "Strongly disagree"]}, "question_22": {"question": "Are you aware of the emotional support services provided by PGS", "answers": ["Not aware", "Aware", "I have used them"]}, "question_23": {"question": "How often have you felt unsafe at work during the last 6 months?", "answers": ["Never", "Almost never", "Sometimes", "Fairly often", "Very often"]}, "question_24": {"question": "The safety culture in my department is good", "answers": ["Strongly agree", "Agree", "Neutral", "Disagree", "Strongly disagree"]}, "question_25": {"question": "I don't have a problem stopping the job if I see anything unsafe", "answers": ["Strongly agree", "Agree", "Neutral", "Disagree", "Strongly disagree"]}, "question_26": {"question": "How often do you feel a rush to complete a job quickly?", "answers": ["Never", "Almost never", "Sometimes", "Fairly often", "Very often"]}, "question_27": {"question": "How often are you distracted or find it difficult to concentrate?", "answers": ["Never", "Almost never", "Sometimes", "Fairly often", "Very often"]}, "question_28": {"question": "On an average day, I usually get the following hours of sleep", "answers": ["0-2", "3-4", "5-6", "7-8", "9-10"]}, "question_29": {"question": "I am able to adapt when changes occur", "answers": ["Not true at all", "Rarely true", "Sometimes true", "Often true", "True nearly all the time"]}, "question_30": {"question": "I tend to bounce back after illness, injury or other hardships", "answers": ["Not true at all", "Rarely true", "Sometimes true", "Often true", "True nearly all the time"]}}
"""

In [73]:
response2 = obtener_respuesta_gpt4(prompt2)
display(Markdown(response2.content))

Para crear una base de datos SQLite a partir del JSON proporcionado, necesitaremos crear tres tablas diferentes: una para la encuesta, una para las preguntas y una para las respuestas. Aquí hay un bosquejo de cómo podemos hacer esto usando Python y la biblioteca SQLite3:

```python
import sqlite3
import json

# Crear una conexión a la base de datos en memoria
conn = sqlite3.connect(':memory:')
c = conn.cursor()

# Crear las tablas
c.execute('''CREATE TABLE survey (id text, name text, date text)''')
c.execute('''CREATE TABLE questions (id text, question text, survey_id text)''')
c.execute('''CREATE TABLE answers (id integer, answer text, question_id text)''')

# Cargar el JSON
data = json.loads('{"id": "000", "name": "anonymous health and wellbeing survey y2 2023", "date": "05-12-2023", "question_0": {"question": "What is your position?", "answers": ["Officer / Chief / SL", "Operator / Rating", "Other"]}, "question_1": {"question": "What is your age?", "answers": ["Under 24", "25 - 34", "35 - 44", "45 - 54", "Over 55"]}, ..., "question_30": {"question": "I tend to bounce back after illness, injury or other hardships", "answers": ["Not true at all", "Rarely true", "Sometimes true", "Often true", "True nearly all the time"]}}')

# Insertar datos en la tabla de encuestas
c.execute("INSERT INTO survey VALUES (?, ?, ?)", (data['id'], data['name'], data['date']))

# Recorrer cada pregunta en los datos
for question_id in range(31):  # Porque hay 31 preguntas desde la pregunta_0 hasta la pregunta_30
    question_data = data[f'question_{question_id}']
    c.execute("INSERT INTO questions VALUES (?, ?, ?)", (f'question_{question_id}', question_data['question'], data['id']))
    # Recorrer cada respuesta en la pregunta
    for answer_id, answer in enumerate(question_data['answers']):
        c.execute("INSERT INTO answers VALUES (?, ?, ?)", (answer_id, answer, f'question_{question_id}'))

# Guardar (commit) los cambios y cerrar la conexión
conn.commit()
conn.close()
```

Este es un bosquejo simple y no maneja posibles errores. En un escenario real, también querrías usar IDs primarios y foráneos para construir relaciones entre las tablas. También tienes que transformar el rango de las preguntas basado en el tamaño de tu json, en este caso son 31 preguntas, del question_0 al question_30, por eso el rango va hasta 31. 

Ten en cuenta también que los datos de las respuestas se guardan por separado, lo que dará alcance a la normalización de los mismos.

In [13]:
response2

'Error al obtener respuesta: Error code: 400 - {\'error\': {\'message\': "\'agent\' is not one of [\'system\', \'assistant\', \'user\', \'function\'] - \'messages.2.role\'", \'type\': \'invalid_request_error\', \'param\': None, \'code\': None}}'

In [74]:
d =        {"survey_id": 0, "name": "anonymous health and wellbeing survey y2 2023", "date": "05-12-2023", 
            0: {"question": "What is your position?", 
                "answers": ["Officer / Chief / SL", "Operator / Rating", "Other"]}, 
            1: {"question": "What is your age?", 
            "answers": ["Under 24", "25 - 34", "35 - 44", "45 - 54", "Over 55"]}}

In [75]:
d

{'survey_id': 0,
 'name': 'anonymous health and wellbeing survey y2 2023',
 'date': '05-12-2023',
 0: {'question': 'What is your position?',
  'answers': ['Officer / Chief / SL', 'Operator / Rating', 'Other']},
 1: {'question': 'What is your age?',
  'answers': ['Under 24', '25 - 34', '35 - 44', '45 - 54', 'Over 55']}}

In [24]:
promp3 = """

podrias hacer que ande mi funcion, me da el singuiente error en la linea que comienza con  messages.extend
'Error al obtener respuesta: Error code: 400 - {\'error\': {\'message\': "\'agent\' is not one of [\'system\', \'assistant\', \'user\', \'function\'] - \'messages.2.role\'", \'type\': \'invalid_request_error\', \'param\': None, \'code\': None}}'
    # Importa la biblioteca de OpenAI
from openai import OpenAI

def obtener_respuesta_gpt4(prompt, historial=[]):
    Realiza una solicitud a la API de OpenAI utilizando el modelo GPT-4.
    
    Args:
        prompt (str): El texto de entrada o pregunta para el modelo.
    
    Returns:
        str: La respuesta generada por el modelo.
    try:
        # Crea una instancia del cliente OpenAI
        cliente = OpenAI()

        messages=[{"role": "user", "content": prompt}]
        if len(historial) > 0:
            # xtend messages list role: changing roles user, agent. the none indexes are agent the pairs are user
            messages.extend([{"role": "user", "content": m} if i % 2 == 0 else {"role": "assistant", "content": m} for i, m in enumerate(historial)])

        # Realiza una solicitud de chat completions con el modelo GPT-4
        respuesta = cliente.chat.completions.create(
            model="gpt-4",  # Utiliza el modelo GPT-4
            messages=messages,
            stream= True
        )

        # Devuelve la respuesta generada por el modelo
        return respuesta.choices[0].message
    except Exception as e:
        return f"Error al obtener respuesta: {str(e)}"                      
"""

In [25]:
response3 = obtener_respuesta_gpt4(promp3)
display(Markdown(response3.content))

Estoy viendo que tratas de cambiar los roles entre 'user' y 'assistant' en la lista de mensajes de historia. Sin embargo, parece que estás empezando con 'user' tanto para el mensaje de solicitud inicial como para el primer mensaje de historia. Eso puede estar causando el error, ya que OpenAI puede esperar que los mensajes se alternen entre 'user' y 'assistant'. Aquí está una versión corregida de tu función que debería solucionar el problema:

```python
from openai import OpenAI

def obtener_respuesta_gpt4(prompt, historial=[]):
    """
    Realiza una solicitud a la API de OpenAI utilizando el modelo GPT-4.
    
    Args:
        prompt (str): El texto de entrada o pregunta para el modelo.
        historial (list, optional): Una lista de mensajes previos del historial de conversación.

    Returns:
        str: La respuesta generada por el modelo.
    """
    try:
        cliente = OpenAI("tu_api_key")

        messages = [{"role": "user", "content": prompt}]
        if len(historial) > 0:
            messages.extend([{"role": "assistant", "content": m}
                             if i % 2 == 0 else {"role": "user", "content": m}
                             for i, m in enumerate(historial)])

        respuesta = cliente.ChatCompletion.create(
            model="text-davinci-002",  # Utiliza el modelo text-davinci-002
            messages=messages,
        )

        return respuesta.choices[0].message['content'] if respuesta.choices else None
    except Exception as e:
        return f"Error al obtener respuesta: {str(e)}"
```

También he hecho algunos cambios importantes:

1. Estás utilizando `openai.OpenAI()` para crear una instancia del cliente, pero deberías utilizar `openai.OpenAI('tu_api_key')`. Asegúrate de reemplazar `'tu_api_key'` con tu propia clave de API de OpenAI.

2. He cambiado `"gpt-4"` a `"text-davinci-002"`, ya que OpenAI no tiene un modelo llamado "GPT-4". Puedes cambiar esto de acuerdo con tu preferencia.

3. He corregido `respuesta.choices[0].message` a `respuesta.choices[0].message['content']` para obtener correctamente el contenido de la respuesta.

4. La clave `'stream': True` ya no está disponible en la versión 0.27.0 de la biblioteca de cliente de OpenAI, por lo que la he retirado.

Estos cambios deberían hacer que tu función funcione correctamente. Asegúrate de seguir las mejores prácticas de manejo de errores para capturar y manejar cualquier error que pueda ocurrir durante la ejecución.

In [20]:
response3

"Error al obtener respuesta: 'Stream' object has no attribute 'choices'"